In [1]:
import datetime as datetime
import pandas as pd
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import requests

short_start = "2019-3-01"
med_start = "2019-1-01"
long_start = "2018-01-01"
short_end = "2019-4-01"
med_end = "2019-4-01"
long_end = "2019-4-01"

drop_list = ['XRP' , 'XLM' , 'BCH' , 'LTC' , 'ADA' , 'TRX' , 'EOS' , 'BSV']
long_drop = ['currency' , 'platform' , 'application']

tfs = ['short' , 'med' , 'long']

df_all = dict.fromkeys(tfs)
df_benchmarks = dict.fromkeys(tfs)
returns_all = dict.fromkeys(tfs)
returns_benchmarks = dict.fromkeys(tfs)
results = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
net_return = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
ann_factor = dict.fromkeys(tfs)
total_return = dict.fromkeys(tfs)
percent_sd = dict.fromkeys(tfs)
summary = dict.fromkeys(tfs)
corr = dict.fromkeys(tfs)
betas = dict.fromkeys(tfs)
beta_results = dict.fromkeys(tfs)
total_return_excel = dict.fromkeys(tfs)
sortino_df = dict.fromkeys(tfs)
sortino = dict.fromkeys(tfs)
average_return = dict.fromkeys(tfs)

data_bletchley = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\\bletchley_history.xlsx' , index_col= 0)
data_coins = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bletchley.xlsx' , index_col=0)
#data_bitcoin = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bitcoin.xlsx' , index_col=0)
data_bletchley['ERC20'] = data_bletchley['ethereum']
data_bletchley = data_bletchley.drop('ethereum' , axis=1)
data_bletch = data_bletchley

In [3]:
data_finance = data_bletch.fillna(method='ffill')
data_finance = data_finance.dropna()
index_data = data_finance
data_coins = data_coins.fillna(method='ffill')
data_coins = data_coins.dropna()

#bitcoin_data = data_bitcoin.fillna(method='ffill')
#bitcoin_data = bitcoin_data.dropna()
#bitcoin_data.index = (pd.to_datetime(bitcoin_data.index.values,unit='ms'))
#bitcoin_data.index = bitcoin_data.index.round('d')

index_data.index = (pd.to_datetime(index_data.index.values,unit='ms'))
index_data.index = index_data.index.round('d')
df = index_data.join(data_coins, how='inner')
#df = df.join(bitcoin_data,how='inner')
df

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
2018-11-08,740.953740,637.900186,110.287135,311.303644,40.569745,27.086623,24.329500,73.888661,6496.04030,216.72468,0.53399,0.25475,53.74036,620.93861,0.02371,5.61640,0.07786,6449.50503
2018-11-09,721.565444,618.797851,106.479184,303.007814,39.834951,25.967035,23.624929,71.826111,6430.36880,210.84548,0.49315,0.25495,52.12370,573.46677,0.02299,5.49671,0.07602,102.66811
2018-11-10,713.346165,609.780474,103.749804,299.376551,39.246942,25.732236,23.221154,70.634206,6401.04440,209.70731,0.50154,0.25714,51.92593,542.51845,0.02313,5.39514,0.07436,70.33582
2018-11-11,716.586940,610.434094,104.335381,300.675133,39.365448,25.905354,23.253192,70.736600,6392.03074,212.19864,0.50669,0.26315,52.17531,553.08937,0.02295,5.41143,0.07513,88.33960
2018-11-12,719.176084,608.269451,102.901564,301.480361,39.486859,26.000422,22.972163,69.884891,6394.35760,211.12914,0.50533,0.27463,51.01492,527.37814,0.02281,5.45015,0.07632,114.76690
2018-11-13,716.724558,611.223194,101.636233,300.550535,39.208126,26.162344,22.709419,68.951585,6352.20546,210.21814,0.51604,0.26608,50.44296,653.26374,0.02228,5.39993,0.07556,726.18475
2018-11-14,708.465071,596.955295,98.718815,296.715644,38.893893,25.491212,22.114844,66.856327,6257.07012,202.78648,0.49017,0.24065,47.80218,472.17701,0.02051,5.10995,0.06918,189.55126
2018-11-15,639.335345,531.505568,86.586428,267.333560,35.057002,22.916454,19.560126,59.560033,5752.48396,183.14469,0.47394,0.23332,43.51497,415.16394,0.01956,4.70250,0.06543,173.57708
2018-11-16,628.338127,520.364846,84.732955,262.641013,34.171254,22.849624,19.211085,58.138167,5617.27551,181.18128,0.47881,0.24039,43.62501,355.38005,0.01893,4.71048,0.06318,89.67520
2018-11-17,612.375185,503.651023,82.562568,255.850709,33.395493,22.059975,18.680848,56.552413,5583.03249,174.92739,0.47398,0.23964,42.12538,295.12364,0.01902,4.58382,0.06147,112.94693


In [4]:
returns = df.pct_change()
returns = returns.dropna()

In [5]:
#filter master dataframes based on date ranges
returns_all['short'] = returns.loc[short_start:short_end]
returns_benchmarks['short'] = returns.loc[short_start:short_end]
df_all['short'] = df.loc[short_start:short_end]
df_benchmarks['short'] = df.loc[short_start:short_end]

returns_all['med'] = returns.loc[med_start:med_end]
returns_benchmarks['med'] = returns.loc[med_start:med_end]
df_all['med'] = df.loc[med_start:med_end]
df_benchmarks['med'] = df.loc[med_start:med_end]

returns_all['long'] = returns.loc[long_start:long_end]
returns_benchmarks['long'] = returns.loc[long_start:long_end]
df_all['long'] = df.loc[long_start:long_end]
df_benchmarks['long'] = df.loc[long_start:long_end]

#drop coins that aren't benchmarks
for c in drop_list:
    for t in tfs:
        returns_benchmarks[t] = returns_benchmarks[t].drop([c] , axis=1)
        df_benchmarks[t] = df_benchmarks[t].drop([c] , axis=1)

In [6]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-23,0.004790,0.012277,0.014354,0.005689,0.002734,0.010890,0.028162,0.033149,0.001757,0.003749
2019-03-24,0.004379,-0.001131,0.021898,0.004500,0.001830,0.011835,0.019018,0.020802,-0.007034,0.004166
2019-03-25,-0.006504,0.022373,0.007289,-0.003758,-0.004081,-0.012389,0.034933,0.039091,-0.004883,-0.013477
2019-03-26,-0.017221,-0.025679,-0.036378,-0.018546,-0.018047,-0.017891,-0.029042,-0.028191,-0.007089,-0.006304
2019-03-27,0.003749,-0.013662,0.005010,0.002379,0.002930,0.006085,-0.011416,-0.011874,0.014617,0.012553


In [7]:
#drop sectors that don't have data back to 1/1/2018
for i in long_drop:
    df_benchmarks['long'] = df_benchmarks['long'].drop([i] , axis=1)
    returns_benchmarks['long'] = returns_benchmarks['long'].drop([i] , axis=1)

In [8]:
#calculate sortino to btc
for t in tfs:
    sortino_df[t] = pd.DataFrame()
    sortino[t] = pd.DataFrame(index = ['sortino'])
    for col in returns_benchmarks[t]:
        sortino_df[t][col] = returns_benchmarks[t][col] - returns_benchmarks[t]['BTC']
        avg_ret = np.mean(sortino_df[t][col])
        a = [r ** 2 for r in sortino_df[t][col] if r < 0]
        a_sum = np.sum(a)
        b = len(sortino_df[t][col])
        down_risk = np.sqrt(a_sum/b)
        if down_risk > 0.000001:
            sortino[t][col] = (avg_ret / down_risk)*math.sqrt(365)
        else:
            sortino[t][col] = 0

In [9]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-23,0.004790,0.012277,0.014354,0.005689,0.002734,0.010890,0.028162,0.033149,0.001757,0.003749
2019-03-24,0.004379,-0.001131,0.021898,0.004500,0.001830,0.011835,0.019018,0.020802,-0.007034,0.004166
2019-03-25,-0.006504,0.022373,0.007289,-0.003758,-0.004081,-0.012389,0.034933,0.039091,-0.004883,-0.013477
2019-03-26,-0.017221,-0.025679,-0.036378,-0.018546,-0.018047,-0.017891,-0.029042,-0.028191,-0.007089,-0.006304
2019-03-27,0.003749,-0.013662,0.005010,0.002379,0.002930,0.006085,-0.011416,-0.011874,0.014617,0.012553


In [10]:
data_coins_week = data_coins.loc[short_start:short_end]
#data_coins_week = data_coins_week[:]
net_week_return = data_coins_week.pct_change(len(data_coins_week)-1)
ann_week_factor = 365.000/(len(data_coins_week))
data_coins_week_returns = pd.DataFrame()
data_coins_week_returns = data_coins_week.pct_change()
data_coins_week_returns = data_coins_week_returns.dropna()
total_week_return = net_week_return.tail(1)*100
data_coins_week = data_coins_week.append(total_week_return)
data_coins_week.index = data_coins_week.index[:-1].append(pd.Index(['return']))

In [11]:
data_coins_week

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
2019-03-01 00:00:00,3811.002120,135.459660,0.313640,0.084510,45.659230,130.066700,0.023680,3.50402,0.042850,66.66179
2019-03-02 00:00:00,3792.679950,135.545760,0.315330,0.084470,47.017210,131.135390,0.023410,3.45225,0.042800,66.41078
2019-03-03 00:00:00,3790.605270,132.253380,0.311760,0.082850,48.205290,129.810270,0.022480,3.45287,0.042140,65.70004
2019-03-04 00:00:00,3747.264700,129.014780,0.304940,0.085570,46.977130,127.791620,0.021940,3.45759,0.041160,63.52825
2019-03-05 00:00:00,3869.656690,128.366380,0.310480,0.085180,47.016910,125.657610,0.023630,3.32773,0.041530,64.51416
2019-03-06 00:00:00,3874.563920,137.063570,0.314370,0.085650,52.577500,131.892570,0.023720,3.75108,0.042810,66.47786
2019-03-07 00:00:00,3860.274510,137.648920,0.316240,0.084600,55.489090,131.299570,0.023190,3.73464,0.042390,66.19705
2019-03-08 00:00:00,3828.126400,136.643840,0.311100,0.084280,56.494440,129.662140,0.022730,3.72927,0.042290,65.88933
2019-03-09 00:00:00,3919.542410,134.729730,0.310370,0.087010,56.178500,128.388460,0.022610,3.63415,0.042690,65.60432
2019-03-10 00:00:00,3900.143380,135.978200,0.309930,0.089110,57.225770,132.064740,0.022490,3.71382,0.046070,65.56212


In [12]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.003810,0.001929
2019-03-02,0.001945,0.033469,0.021862,0.004711,0.003317,0.005152,0.032925,0.036816,-0.004808,0.000636
2019-03-03,-0.002430,-0.000707,-0.000303,-0.002244,0.001227,-0.009133,0.001460,0.000615,-0.000547,-0.024290
2019-03-04,-0.011021,-0.021609,-0.008489,-0.011707,-0.009177,-0.013162,-0.019840,-0.021733,-0.011434,-0.024488
2019-03-05,-0.024378,-0.029798,-0.037056,-0.025141,-0.021542,-0.042697,-0.015249,-0.012223,0.032662,-0.005026
2019-03-06,0.052274,0.084908,0.043091,0.054320,0.042553,0.079849,0.088721,0.095061,0.001268,0.067753
2019-03-07,0.002188,0.014242,-0.004632,0.002867,0.002098,0.000729,0.012888,0.015896,-0.003688,0.004271
2019-03-08,0.002430,0.023899,0.030511,0.004814,0.004598,0.006727,0.027980,0.021305,-0.008328,-0.007302
2019-03-09,-0.011954,-0.031500,0.002758,-0.012987,-0.007684,-0.024343,-0.023665,-0.028403,0.023880,-0.014008
2019-03-10,0.021451,0.028492,0.048644,0.022784,0.019861,0.033830,0.034807,0.031074,-0.004949,0.009266


In [13]:
print ('short start: ', returns_benchmarks['short'].index[0])
print ('medium start: ', returns_benchmarks['med'].index[0])
print ('long start: ', returns_benchmarks['long'].index[0])
print ('end: ', returns_benchmarks['short'].index[-2])
print ('end: ', returns_benchmarks['med'].index[-2])
print ('end: ', returns_benchmarks['long'].index[-2])

short start:  2019-03-01 00:00:00
medium start:  2019-01-01 00:00:00
long start:  2018-11-09 00:00:00
end:  2019-03-26 00:00:00
end:  2019-03-26 00:00:00
end:  2019-03-26 00:00:00


In [14]:
returns_benchmarks['short'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.003810,0.001929
2019-03-02,0.001945,0.033469,0.021862,0.004711,0.003317,0.005152,0.032925,0.036816,-0.004808,0.000636
2019-03-03,-0.002430,-0.000707,-0.000303,-0.002244,0.001227,-0.009133,0.001460,0.000615,-0.000547,-0.024290
2019-03-04,-0.011021,-0.021609,-0.008489,-0.011707,-0.009177,-0.013162,-0.019840,-0.021733,-0.011434,-0.024488
2019-03-05,-0.024378,-0.029798,-0.037056,-0.025141,-0.021542,-0.042697,-0.015249,-0.012223,0.032662,-0.005026


In [15]:
print (data_coins_week.head(n=1))
data_coins_week.tail(n=1)

                            BTC        ETH      XRP      XLM       LTC  \
2019-03-01 00:00:00  3811.00212  135.45966  0.31364  0.08451  45.65923   

                          BCH      TRX      EOS      ADA       BSV  
2019-03-01 00:00:00  130.0667  0.02368  3.50402  0.04285  66.66179  


,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,4.614314,0.038041,-1.960847,24.470477,30.862588,24.148941,-2.913851,9.1669,47.234539,-2.11898


In [16]:
annualize = math.sqrt(365)
for t in tfs:
    results[t] = returns_benchmarks[t].describe()
    vol[t] = pd.DataFrame()
    vol[t] = results[t].loc['std']*annualize*100
    vol[t].name = 'annualized volatility'
    results[t] = results[t].append(df_benchmarks[t].iloc[0])
    results[t].index = results[t].index[:-1].append(pd.Index(['open price']))
    results[t] = results[t].append(df_benchmarks[t][-1:])
    results[t].index = results[t].index[:-1].append(pd.Index(['last price']))
    results[t] = results[t].append(vol[t])

In [17]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-23,0.004790,0.012277,0.014354,0.005689,0.002734,0.010890,0.028162,0.033149,0.001757,0.003749
2019-03-24,0.004379,-0.001131,0.021898,0.004500,0.001830,0.011835,0.019018,0.020802,-0.007034,0.004166
2019-03-25,-0.006504,0.022373,0.007289,-0.003758,-0.004081,-0.012389,0.034933,0.039091,-0.004883,-0.013477
2019-03-26,-0.017221,-0.025679,-0.036378,-0.018546,-0.018047,-0.017891,-0.029042,-0.028191,-0.007089,-0.006304
2019-03-27,0.003749,-0.013662,0.005010,0.002379,0.002930,0.006085,-0.011416,-0.011874,0.014617,0.012553


In [18]:
for t in tfs:
    average_return[t] = pd.DataFrame()
    average_return[t] = returns_benchmarks[t]
    average_return[t].loc['mean'] = returns_benchmarks[t].mean(axis=0)
    returns_benchmarks[t] = returns_benchmarks[t][:-1]

In [19]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-01 00:00:00,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.003810,0.001929
2019-03-02 00:00:00,0.001945,0.033469,0.021862,0.004711,0.003317,0.005152,0.032925,0.036816,-0.004808,0.000636
2019-03-03 00:00:00,-0.002430,-0.000707,-0.000303,-0.002244,0.001227,-0.009133,0.001460,0.000615,-0.000547,-0.024290
2019-03-04 00:00:00,-0.011021,-0.021609,-0.008489,-0.011707,-0.009177,-0.013162,-0.019840,-0.021733,-0.011434,-0.024488
2019-03-05 00:00:00,-0.024378,-0.029798,-0.037056,-0.025141,-0.021542,-0.042697,-0.015249,-0.012223,0.032662,-0.005026
2019-03-06 00:00:00,0.052274,0.084908,0.043091,0.054320,0.042553,0.079849,0.088721,0.095061,0.001268,0.067753
2019-03-07 00:00:00,0.002188,0.014242,-0.004632,0.002867,0.002098,0.000729,0.012888,0.015896,-0.003688,0.004271
2019-03-08 00:00:00,0.002430,0.023899,0.030511,0.004814,0.004598,0.006727,0.027980,0.021305,-0.008328,-0.007302
2019-03-09 00:00:00,-0.011954,-0.031500,0.002758,-0.012987,-0.007684,-0.024343,-0.023665,-0.028403,0.023880,-0.014008
2019-03-10 00:00:00,0.021451,0.028492,0.048644,0.022784,0.019861,0.033830,0.034807,0.031074,-0.004949,0.009266


In [20]:
for t in tfs:
    net_return[t] = df_benchmarks[t].pct_change(len(df_benchmarks[t])-1)
    ann_factor[t] = 365.000/(len(df_benchmarks[t]))
    total_return[t] = net_return[t].tail(1)*100
    total_return_excel[t] = net_return[t].tail(1)
    percent_sd[t] = results[t].iloc[2,:]*100
    results[t] = results[t].append(percent_sd[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['daily volatility']))
    #print total_return_excel[t]
    results[t] = results[t].append(total_return_excel[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['return']))
    #print results[t]
    #print average_return[t]/(percent_sd[t]*0.01).transpose()
    Sharpe_values = (average_return[t].loc['mean']/(percent_sd[t]*0.01))*annualize
    Sharpe_values = Sharpe_values.rename('sharpe_values')
    results[t] = results[t].append(Sharpe_values)
    results[t].index = results[t].index[:-1].append(pd.Index(['Sharpe']))
    results[t] = results[t].append(sortino[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['Sortino']))
#print (results[t])

In [21]:
returns_benchmarks['med'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-01 00:00:00,-0.046381,-0.050800,-0.056148,-0.046939,-0.042739,-0.057864,-0.040122,-0.046808,-0.014056,-0.038308
2019-01-02 00:00:00,0.046762,0.043521,0.042677,0.046447,0.043137,0.051576,0.036109,0.039667,0.046970,0.058378
2019-01-03 00:00:00,0.033371,0.035948,0.027412,0.033396,0.022256,0.053596,0.031989,0.027348,-0.018288,0.083127
2019-01-04 00:00:00,-0.041354,-0.043182,-0.034871,-0.041317,-0.036859,-0.054475,-0.028133,-0.029247,-0.002285,-0.028170
2019-01-05 00:00:00,0.018170,0.025959,0.024532,0.018870,0.013260,0.025122,0.048155,0.050412,0.003006,0.040165


In [22]:
for t in tfs:
    summary[t] = results[t].iloc[8:]
    corr[t] = returns_benchmarks[t].corr(method='spearman')
    betas[t] = returns_benchmarks[t].cov()
    beta_results[t] = pd.DataFrame()
    beta_results[t] = betas[t].iloc[0,:] / betas[t].iloc[3,0]
    corr[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_corr'+t+'.csv')
    summary[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_results'+t+'.csv')
beta_all = pd.DataFrame()
beta_all['MTD'] = beta_results['short']
beta_all['QTD'] = beta_results['med']
beta_all['YTD'] = beta_results['long']

In [23]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-23 00:00:00,0.004790,0.012277,0.014354,0.005689,0.002734,0.010890,0.028162,0.033149,0.001757,0.003749
2019-03-24 00:00:00,0.004379,-0.001131,0.021898,0.004500,0.001830,0.011835,0.019018,0.020802,-0.007034,0.004166
2019-03-25 00:00:00,-0.006504,0.022373,0.007289,-0.003758,-0.004081,-0.012389,0.034933,0.039091,-0.004883,-0.013477
2019-03-26 00:00:00,-0.017221,-0.025679,-0.036378,-0.018546,-0.018047,-0.017891,-0.029042,-0.028191,-0.007089,-0.006304
2019-03-27 00:00:00,0.003749,-0.013662,0.005010,0.002379,0.002930,0.006085,-0.011416,-0.011874,0.014617,0.012553


In [24]:
summary['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,424.914158,381.398019,57.301770,178.648490,22.516762,15.764553,17.515368,50.875261,3811.002120,135.459660
last price,440.071152,459.772825,67.614739,187.877919,23.613672,16.932204,21.080038,62.032900,3986.853710,135.511190
annualized volatility,29.768297,50.237837,42.771735,30.826303,25.932167,44.021748,50.921672,54.362797,20.101147,36.672250
daily volatility,1.558144,2.629569,2.238775,1.613522,1.357352,2.304204,2.665362,2.845479,1.052142,1.919513
return,0.035671,0.205493,0.179976,0.051663,0.048715,0.074068,0.203517,0.219314,0.046143,0.000380
Sharpe,1.738057,5.313688,5.539676,2.367788,2.543780,2.415253,5.561313,5.333098,2.874907,0.257984
Sortino,-0.219943,5.944120,5.526767,0.542237,0.320358,1.353651,6.911117,7.002968,0.000000,-1.920088


In [25]:
summary['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,420.511599,337.862280,54.644247,175.290083,22.237102,15.347474,13.168780,39.924454,3769.486990,134.022370
last price,440.071152,459.772825,67.614739,187.877919,23.613672,16.932204,21.080038,62.032900,3986.853710,135.511190
annualized volatility,57.126773,66.773618,61.270566,57.359844,51.738642,73.417554,65.994769,68.683633,33.937508,72.244722
daily volatility,2.990152,3.495091,3.207048,3.002351,2.708124,3.842850,3.454324,3.595066,1.776370,3.781462
return,0.046514,0.360829,0.237362,0.071811,0.061904,0.103257,0.600759,0.553757,0.057665,0.011109
Sharpe,0.269472,1.980592,1.396761,0.445292,0.393267,0.593237,3.128532,2.799194,0.698175,0.192409
Sortino,-0.243064,2.726539,1.684338,0.053819,-0.109778,0.443187,4.968077,4.441638,0.000000,-0.239774


In [26]:
summary['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
open price,740.953740,637.900186,110.287135,311.303644,73.888661,6496.040300,216.724680
last price,440.071152,459.772825,67.614739,187.877919,62.032900,3986.853710,135.511190
annualized volatility,78.035051,90.084420,91.559281,78.814472,92.979831,58.925156,90.666608
daily volatility,4.084541,4.715234,4.792432,4.125338,4.866787,3.084284,4.745707
return,-0.406075,-0.279240,-0.386921,-0.396480,-0.160454,-0.386264,-0.374731
Sharpe,-1.371466,-0.506133,-0.947021,-1.296231,-0.029556,-1.890737,-0.917976
Sortino,0.139400,1.681390,0.621044,0.290460,2.763283,0.000000,0.693596


In [27]:
corr['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.782662,0.801587,0.988400,0.978022,0.976801,0.725885,0.731380,0.072039,0.810134
twenty,0.782662,1.000000,0.783883,0.840659,0.813187,0.796703,0.890720,0.901099,-0.218559,0.510379
forty,0.801587,0.783883,1.000000,0.833944,0.811355,0.835775,0.794261,0.774115,-0.109280,0.527473
total,0.988400,0.840659,0.833944,1.000000,0.984127,0.974969,0.770452,0.777167,0.024420,0.772894
currency,0.978022,0.813187,0.811355,0.984127,1.000000,0.951160,0.711233,0.724664,0.081807,0.762515
platform,0.976801,0.796703,0.835775,0.974969,0.951160,1.000000,0.717338,0.719780,-0.018315,0.789377
application,0.725885,0.890720,0.794261,0.770452,0.711233,0.717338,1.000000,0.985348,-0.231380,0.521978
ERC20,0.731380,0.901099,0.774115,0.777167,0.724664,0.719780,0.985348,1.000000,-0.197192,0.538462
BTC,0.072039,-0.218559,-0.109280,0.024420,0.081807,-0.018315,-0.231380,-0.197192,1.000000,0.311355
ETH,0.810134,0.510379,0.527473,0.772894,0.762515,0.789377,0.521978,0.538462,0.311355,1.000000


In [28]:
corr['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.868497,0.867891,0.996013,0.992242,0.973793,0.796658,0.789486,0.261853,0.818448
twenty,0.868497,1.000000,0.818722,0.898007,0.867481,0.903635,0.927360,0.926324,0.223451,0.700723
forty,0.867891,0.818722,1.000000,0.884659,0.864041,0.892789,0.803127,0.774731,0.257416,0.724565
total,0.996013,0.898007,0.884659,1.000000,0.989154,0.979148,0.831972,0.823862,0.254368,0.816201
currency,0.992242,0.867481,0.864041,0.989154,1.000000,0.954251,0.795642,0.787395,0.276490,0.795212
platform,0.973793,0.903635,0.892789,0.979148,0.954251,1.000000,0.814579,0.805159,0.234415,0.829353
application,0.796658,0.927360,0.803127,0.831972,0.795642,0.814579,1.000000,0.987043,0.247782,0.655814
ERC20,0.789486,0.926324,0.774731,0.823862,0.787395,0.805159,0.987043,1.000000,0.236701,0.651202
BTC,0.261853,0.223451,0.257416,0.254368,0.276490,0.234415,0.247782,0.236701,1.000000,0.349111
ETH,0.818448,0.700723,0.724565,0.816201,0.795212,0.829353,0.655814,0.651202,0.349111,1.000000


In [29]:
corr['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
ten,1.000000,0.909148,0.919313,0.997941,0.858206,0.613598,0.841051
twenty,0.909148,1.000000,0.899404,0.927414,0.945761,0.558729,0.801223
forty,0.919313,0.899404,1.000000,0.931692,0.868805,0.614570,0.814740
total,0.997941,0.927414,0.931692,1.000000,0.877088,0.608593,0.843736
ERC20,0.858206,0.945761,0.868805,0.877088,1.000000,0.573309,0.745619
BTC,0.613598,0.558729,0.614570,0.608593,0.573309,1.000000,0.557446
ETH,0.841051,0.801223,0.814740,0.843736,0.745619,0.557446,1.000000


In [30]:
beta_all

,MTD,QTD,YTD
ten,0.968961,0.996887,0.990551
twenty,1.372625,1.052954,1.089356
forty,1.033304,0.954230,1.100921
total,1.000000,1.000000,1.000000
currency,0.836865,0.897143,NaN
platform,1.384539,1.258174,NaN
application,1.350914,0.943399,NaN
ERC20,1.421143,0.958144,1.064842
BTC,-0.095028,0.290815,0.602170
ETH,1.013996,1.114227,1.017095


In [31]:
coins_high_low = data_coins_week_returns.describe()
coins_high_low

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,0.001791,0.000195,-0.000699,0.009025,0.010856,0.008833,-0.000755,0.003834,0.015741,-0.000702
std,0.010673,0.019572,0.011376,0.034948,0.031451,0.032005,0.028446,0.031539,0.040172,0.015911
min,-0.011434,-0.024773,-0.021876,-0.041852,-0.027081,-0.030897,-0.051452,-0.037558,-0.038742,-0.033056
25%,-0.006064,-0.012013,-0.005996,-0.015602,-0.013155,-0.014430,-0.018557,-0.014720,-0.011485,-0.009261
50%,-0.000898,-0.003383,-0.001579,0.002522,0.008411,-0.005538,-0.004191,-0.001176,0.006576,-0.004275
75%,0.006412,0.007669,0.005088,0.029059,0.021026,0.026524,0.005394,0.012661,0.040401,0.004773
max,0.032662,0.067753,0.020733,0.108293,0.118268,0.086922,0.077028,0.127219,0.103132,0.030438


In [32]:
returns_benchmarks['short'].describe()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.001418,0.007314,0.006492,0.002000,0.001807,0.002913,0.007759,0.007943,0.001583,0.000259
std,0.015581,0.026296,0.022388,0.016135,0.013574,0.023042,0.026654,0.028455,0.010521,0.019195
min,-0.024378,-0.031500,-0.037056,-0.025141,-0.021542,-0.042697,-0.032617,-0.032548,-0.011434,-0.024773
25%,-0.008577,-0.010955,-0.005632,-0.007159,-0.005882,-0.011538,-0.011496,-0.012049,-0.005692,-0.011648
50%,0.001013,0.001660,0.005822,0.001961,0.001467,0.000729,0.006975,0.004861,-0.001248,-0.002126
75%,0.005013,0.021192,0.021880,0.006893,0.005784,0.013542,0.027667,0.026190,0.005728,0.006536
max,0.052274,0.084908,0.048644,0.054320,0.042553,0.079849,0.088721,0.095061,0.032662,0.067753


In [33]:
data_coins_week.tail(n=1)

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,4.614314,0.038041,-1.960847,24.470477,30.862588,24.148941,-2.913851,9.1669,47.234539,-2.11898
